# Pandas 分组与聚合

## 层级索引（hierarchical indexing）

常用于分组操作，透视表的生成等

In [1]:
import pandas as pd
import numpy as np

In [2]:
ser_obj = pd.Series(np.random.randn(12),index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c', 'd', 'd', 'd'],
                           [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]])
print(ser_obj)

a  0   -2.473549
   1    0.895329
   2    1.239202
b  0    0.023345
   1    0.597253
   2    0.753962
c  0    0.591192
   1    1.252330
   2    3.507989
d  0    0.560484
   1   -0.363496
   2    0.811888
dtype: float64


### MultiIndex索引对象

分为内层与外层索引

In [3]:
print(type(ser_obj.index))
print(ser_obj.index)

<class 'pandas.core.indexes.multi.MultiIndex'>
MultiIndex(levels=[['a', 'b', 'c', 'd'], [0, 1, 2]],
           labels=[[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]])


### 选取子集

In [17]:
# 外层选取
print(ser_obj['c'])

0    0.591192
1    1.252330
2    3.507989
dtype: float64


In [5]:
# 内层选取
print(ser_obj[:, 2])

a    1.239202
b    0.753962
c    3.507989
d    0.811888
dtype: float64


### 层级操作

In [6]:
# 交换分层顺序
print(ser_obj.swaplevel())

0  a   -2.473549
1  a    0.895329
2  a    1.239202
0  b    0.023345
1  b    0.597253
2  b    0.753962
0  c    0.591192
1  c    1.252330
2  c    3.507989
0  d    0.560484
1  d   -0.363496
2  d    0.811888
dtype: float64


In [8]:
# 交换并排序分层
print(ser_obj.swaplevel().sort_index())

0  a   -2.473549
   b    0.023345
   c    0.591192
   d    0.560484
1  a    0.895329
   b    0.597253
   c    1.252330
   d   -0.363496
2  a    1.239202
   b    0.753962
   c    3.507989
   d    0.811888
dtype: float64


##  分组 GroupBy

### GroupBy 对象

In [9]:
import pandas as pd
import numpy as np

In [10]:
dict_obj = {'key1':['a', 'b', 'a', 'b', 
                      'a', 'b', 'a', 'a'],
           'key2' : ['one', 'one', 'two', 'three',
                      'two', 'two', 'one', 'three'],
            'data1': np.random.randn(8),
            'data2': np.random.randn(8)}
df_obj = pd.DataFrame(dict_obj)
print(df_obj)

      data1     data2 key1   key2
0  0.362332  0.672696    a    one
1  0.151453  0.960367    b    one
2 -0.630200  1.185972    a    two
3  1.191360 -0.474772    b  three
4 -1.417819  1.107364    a    two
5 -0.047919  0.712258    b    two
6 -0.216123  0.094480    a    one
7  2.269779 -0.276750    a  three


In [11]:
# dataframe根据key1进行分组
print(type(df_obj.groupby('key1')))

<class 'pandas.core.groupby.DataFrameGroupBy'>


In [12]:
# data1列根据key1进行分组
print(type(df_obj['data1'].groupby(df_obj['key1'])))

<class 'pandas.core.groupby.SeriesGroupBy'>


In [16]:
# 分组运算
grouped1 = df_obj.groupby('key1')
print(grouped1.mean())

grouped2 = df_obj['data1'].groupby(df_obj['key1'])
print(grouped2.mean())

         data1     data2
key1                    
a     0.073594  0.556752
b     0.431631  0.399285
key1
a    0.073594
b    0.431631
Name: data1, dtype: float64


In [21]:
# 按自定义key分组，列表

self_def_key = [1, 1, 2, 2, 2, 1, 1, 1]
df_obj.groupby(self_def_key).size()

1    5
2    3
dtype: int64

In [22]:
# 按自定义key分组，多层列表

df_obj.groupby([df_obj['key1'],df_obj['key2']]).size()

key1  key2 
a     one      2
      three    1
      two      2
b     one      1
      three    1
      two      1
dtype: int64

In [23]:
# 按多个列多层分组
grouped2 = df_obj.groupby(['key1', 'key2'])
print(grouped2.size())

key1  key2 
a     one      2
      three    1
      two      2
b     one      1
      three    1
      two      1
dtype: int64


In [24]:
# 多层分组按key的顺序进行
grouped3 = df_obj.groupby(['key2', 'key1'])
print(grouped3.mean())
print()
print(grouped3.mean().unstack())

               data1     data2
key2  key1                    
one   a     0.073105  0.383588
      b     0.151453  0.960367
three a     2.269779 -0.276750
      b     1.191360 -0.474772
two   a    -1.024009  1.146668
      b    -0.047919  0.712258

          data1               data2          
key1          a         b         a         b
key2                                         
one    0.073105  0.151453  0.383588  0.960367
three  2.269779  1.191360 -0.276750 -0.474772
two   -1.024009 -0.047919  1.146668  0.712258


### GroupBy对象分组迭代

In [27]:
# 单层分组
for group_name, group_data in grouped1:
    print(group_name)
    print(group_data)

a
      data1     data2 key1   key2
0  0.362332  0.672696    a    one
2 -0.630200  1.185972    a    two
4 -1.417819  1.107364    a    two
6 -0.216123  0.094480    a    one
7  2.269779 -0.276750    a  three
b
      data1     data2 key1   key2
1  0.151453  0.960367    b    one
3  1.191360 -0.474772    b  three
5 -0.047919  0.712258    b    two


In [28]:
# 多层分组
for group_name, group_data in grouped2:
    print(group_name)
    print(group_data)

('a', 'one')
      data1     data2 key1 key2
0  0.362332  0.672696    a  one
6 -0.216123  0.094480    a  one
('a', 'three')
      data1    data2 key1   key2
7  2.269779 -0.27675    a  three
('a', 'two')
      data1     data2 key1 key2
2 -0.630200  1.185972    a  two
4 -1.417819  1.107364    a  two
('b', 'one')
      data1     data2 key1 key2
1  0.151453  0.960367    b  one
('b', 'three')
     data1     data2 key1   key2
3  1.19136 -0.474772    b  three
('b', 'two')
      data1     data2 key1 key2
5 -0.047919  0.712258    b  two


In [29]:
# GroupBy对象转换list
list(grouped1)

[('a',       data1     data2 key1   key2
  0  0.362332  0.672696    a    one
  2 -0.630200  1.185972    a    two
  4 -1.417819  1.107364    a    two
  6 -0.216123  0.094480    a    one
  7  2.269779 -0.276750    a  three), ('b',       data1     data2 key1   key2
  1  0.151453  0.960367    b    one
  3  1.191360 -0.474772    b  three
  5 -0.047919  0.712258    b    two)]

In [32]:
# GroupBy对象转换dict
dict(list(grouped1))

{'a':       data1     data2 key1   key2
 0  0.362332  0.672696    a    one
 2 -0.630200  1.185972    a    two
 4 -1.417819  1.107364    a    two
 6 -0.216123  0.094480    a    one
 7  2.269779 -0.276750    a  three, 'b':       data1     data2 key1   key2
 1  0.151453  0.960367    b    one
 3  1.191360 -0.474772    b  three
 5 -0.047919  0.712258    b    two}

In [37]:
# 按数据类型分组
print(df_obj.dtypes)
print()

print(df_obj.groupby(df_obj.dtypes, axis=1).size())
df_obj.groupby(df_obj.dtypes, axis=1).sum()

data1    float64
data2    float64
key1      object
key2      object
dtype: object

float64    2
object     2
dtype: int64


,float64,object
0,1.035029,aone
1,1.111820,bone
2,0.555772,atwo
3,0.716588,bthree
4,-0.310455,atwo
5,0.664339,btwo
6,-0.121643,aone
7,1.993029,athree


### 其他分组方法

In [39]:
df_obj2 = pd.DataFrame(np.random.randint(1, 10, (5,5)),
                       columns=['a', 'b', 'c', 'd', 'e'],
                       index=['A', 'B', 'C', 'D', 'E'])
df_obj2.iloc[1, 1:4] = np.NaN
df_obj2

,a,b,c,d,e
A,7,4.0,9.0,7.0,5
B,3,NaN,NaN,NaN,6
C,9,8.0,4.0,3.0,7
D,6,7.0,9.0,4.0,1
E,5,3.0,4.0,2.0,3


In [42]:
# 通过字典分组
mapping_dict = {'a':'python', 'b':'python', 'c':'java', 'd':'C', 'e':'java'}
print(df_obj2.groupby(mapping_dict, axis=1).size())
print(df_obj2.groupby(mapping_dict, axis=1).count()) # 非NaN的个数
print(df_obj2.groupby(mapping_dict, axis=1).sum())

C         1
java      2
python    2
dtype: int64
   C  java  python
A  1     2       2
B  0     1       1
C  1     2       2
D  1     2       2
E  1     2       2
     C  java  python
A  7.0  14.0    11.0
B  NaN   6.0     3.0
C  3.0  11.0    17.0
D  4.0  10.0    13.0
E  2.0   7.0     8.0


In [49]:
print(df_obj2.groupby('a', axis=0).sum())
print(df_obj2.groupby('a', axis=0).sum().reset_index())

     b    c    d  e
a                  
3  NaN  NaN  NaN  6
5  3.0  4.0  2.0  3
6  7.0  9.0  4.0  1
7  4.0  9.0  7.0  5
9  8.0  4.0  3.0  7
   a    b    c    d  e
0  3  NaN  NaN  NaN  6
1  5  3.0  4.0  2.0  3
2  6  7.0  9.0  4.0  1
3  7  4.0  9.0  7.0  5
4  9  8.0  4.0  3.0  7


In [52]:
# 通过函数分组
df_obj3 = pd.DataFrame(np.random.randint(1, 10, (5,5)),
                       columns=['a', 'b', 'c', 'd', 'e'],
                       index=['AA', 'BBB', 'CC', 'D', 'EE'])
def group_key(idx):
    """
        idx 为列索引或行索引
    """
    #return idx
    return len(idx)

df_obj3.groupby(group_key).size()

# 以上自定义函数等价于
#df_obj3.groupby(len).size()

1    1
2    3
3    1
dtype: int64

In [53]:
# 通过索引级别分组
columns = pd.MultiIndex.from_arrays([['Python', 'Java', 'Python', 'Java', 'Python'],
                                     ['A', 'A', 'B', 'C', 'B']], names=['language', 'index'])
df_obj4 = pd.DataFrame(np.random.randint(1, 10, (5, 5)), columns=columns)
df_obj4

language,Python,Java,Python,Java,Python
index,A,A,B,C,B
0,6,5,7,4,7
1,1,3,5,3,6
2,2,3,6,5,6
3,2,8,5,7,4
4,4,9,9,3,5


In [54]:
# 根据language进行分组
print(df_obj4.groupby(level='language', axis=1).sum())
print(df_obj4.groupby(level='index', axis=1).sum())

language  Java  Python
0            9      20
1            6      12
2            8      14
3           15      11
4           12      18
index   A   B  C
0      11  14  4
1       4  11  3
2       5  12  5
3      10   9  7
4      13  14  3


## 聚合

In [55]:
dict_obj = {'key1' : ['a', 'b', 'a', 'b', 
                      'a', 'b', 'a', 'a'],
            'key2' : ['one', 'one', 'two', 'three',
                      'two', 'two', 'one', 'three'],
            'data1': np.random.randint(1,10, 8),
            'data2': np.random.randint(1,10, 8)}
df_obj5 = pd.DataFrame(dict_obj)
print(df_obj5)

   data1  data2 key1   key2
0      2      8    a    one
1      7      3    b    one
2      5      9    a    two
3      3      3    b  three
4      1      8    a    two
5      6      8    b    two
6      1      9    a    one
7      2      3    a  three


In [57]:
# 内置的聚合函数
print(df_obj5.groupby('key1').sum())
print(df_obj5.groupby('key1').max())
print(df_obj5.groupby('key1').min())
print(df_obj5.groupby('key1').mean())
print(df_obj5.groupby('key1').size())
print(df_obj5.groupby('key1').count())
print(df_obj5.groupby('key1').describe())

      data1  data2
key1              
a        11     37
b        16     14
      data1  data2 key2
key1                   
a         5      9  two
b         7      8  two
      data1  data2 key2
key1                   
a         1      3  one
b         3      3  one
         data1     data2
key1                    
a     2.200000  7.400000
b     5.333333  4.666667
key1
a    5
b    3
dtype: int64
      data1  data2  key2
key1                    
a         5      5     5
b         3      3     3
     data1                                              data2            \
     count      mean       std  min  25%  50%  75%  max count      mean   
key1                                                                      
a      5.0  2.200000  1.643168  1.0  1.0  2.0  2.0  5.0   5.0  7.400000   
b      3.0  5.333333  2.081666  3.0  4.5  6.0  6.5  7.0   3.0  4.666667   

                                         
           std  min  25%  50%  75%  max  
key1                                    

In [59]:
# 自定义的聚合函数
def peak_range(df):
    """
      返回数值范围
    """
    return df.max() - df.min()

print(df_obj5.groupby('key1').agg(peak_range))
print(df_obj5.groupby('key1').agg(lambda df : df.max() - df.min()))

      data1  data2
key1              
a         4      6
b         4      5
      data1  data2
key1              
a         4      6
b         4      5
